In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from aicsshparam import shtools

# Load your data
path = '/net/beliveau/vol2/instrument/E9.5_290/Zoom_290_subset_test/FIRST_ALIGNED_SUCCESS.csv'
df = pd.read_csv(path)

def get_coeffs_from_row(row, lmax=4):
    """Convert CSV row coefficients to shtools format"""
    coeffs = np.zeros((2, lmax+1, lmax+1))
    
    # Cosine coefficients (C)
    for l in range(lmax+1):
        for m in range(l+1):
            coeffs[0, l, m] = row[f'shcoeffs_L{l}M{m}C']
    
    # Sine coefficients (S)
    for l in range(lmax+1):
        for m in range(1, l+1):
            coeffs[1, l, m] = row[f'shcoeffs_L{l}M{m}S']
    
    return coeffs

def visualize_cells(df, cell_indices=[0, 1, 2], lmax=8):
    """Visualize reconstructed cells from spherical harmonic coefficients"""
    
    fig = plt.figure(figsize=(15, 5))
    
    for i, cell_idx in enumerate(cell_indices):
        # Get coefficients for this cell
        row = df.iloc[cell_idx]
        coeffs = get_coeffs_from_row(row, lmax)
        
        # Reconstruct mesh from coefficients
        mesh = shtools.get_reconstruction_from_coeffs(coeffs)
        
        # Plot 3D mesh
        ax = fig.add_subplot(1, len(cell_indices), i+1, projection='3d')
        ax.plot_trisurf(mesh[:, 0], mesh[:, 1], mesh[:, 2], 
                       alpha=0.7, cmap='viridis')
        
        ax.set_title(f'Cell {row["label"]} (lmax={lmax})')
        ax.set_xlabel('X')
        ax.set_ylabel('Y') 
        ax.set_zlabel('Z')
        
        # Make axes equal
        max_range = np.array([mesh[:, 0].max()-mesh[:, 0].min(),
                             mesh[:, 1].max()-mesh[:, 1].min(),
                             mesh[:, 2].max()-mesh[:, 2].min()]).max() / 2.0
        mid_x = (mesh[:, 0].max()+mesh[:, 0].min()) * 0.5
        mid_y = (mesh[:, 1].max()+mesh[:, 1].min()) * 0.5
        mid_z = (mesh[:, 2].max()+mesh[:, 2].min()) * 0.5
        ax.set_xlim(mid_x - max_range, mid_x + max_range)
        ax.set_ylim(mid_y - max_range, mid_y + max_range)
        ax.set_zlim(mid_z - max_range, mid_z + max_range)
    
    plt.tight_layout()
    plt.show()

def compare_lmax_reconstructions(df, cell_idx=0, lmax_values=[2, 4, 6, 8]):
    """Compare reconstruction quality at different lmax values"""
    
    fig = plt.figure(figsize=(20, 5))
    row = df.iloc[cell_idx]
    
    for i, lmax in enumerate(lmax_values):
        # Get coefficients up to lmax
        coeffs = get_coeffs_from_row(row, lmax)
        
        # Reconstruct mesh
        mesh = shtools.get_reconstruction_from_coeffs(coeffs, lrec=lmax)
        
        # Plot
        ax = fig.add_subplot(1, len(lmax_values), i+1, projection='3d')
        ax.plot_trisurf(mesh[:, 0], mesh[:, 1], mesh[:, 2], 
                       alpha=0.7, cmap='plasma')
        
        ax.set_title(f'Cell {row["label"]}\nlmax={lmax}')
        ax.set_xlabel('X')
        ax.set_ylabel('Y')
        ax.set_zlabel('Z')
        
        # Equal aspect ratio
        max_range = np.array([mesh[:, 0].max()-mesh[:, 0].min(),
                             mesh[:, 1].max()-mesh[:, 1].min(),
                             mesh[:, 2].max()-mesh[:, 2].min()]).max() / 2.0
        mid_x = (mesh[:, 0].max()+mesh[:, 0].min()) * 0.5
        mid_y = (mesh[:, 1].max()+mesh[:, 1].min()) * 0.5
        mid_z = (mesh[:, 2].max()+mesh[:, 2].min()) * 0.5
        ax.set_xlim(mid_x - max_range, mid_x + max_range)
        ax.set_ylim(mid_y - max_range, mid_y + max_range)
        ax.set_zlim(mid_z - max_range, mid_z + max_range)
    
    plt.tight_layout()
    plt.show()

# Usage examples:

# Visualize first 3 cells with interactive 3D
visualize_cells(df, cell_indices=[0, 1, 2])


KeyError: 'shcoeffs_L5M0C'

<Figure size 1500x500 with 0 Axes>